# SSD: Single Shot MultiBox Detector

This is designed to visualize priors (*a.k.a* anchors).

### 1. Setup
* First, set up Python, numpy and matplotlib

In [1]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# display plots in this notebook
%matplotlib inline

# set display defaults
plt.rcParams['figure.figsize'] = (12, 12)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

* Load caffe

In [2]:
# The caffe module needs to be on the Python path;
#  we'll add it here explicitly.
import sys
caffe_root = '../../../'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')

import caffe
# If you get "No module named _caffe", either you have not built pycaffe or you have the wrong path.

### 2. Load net and set up input preprocessing
* Set caffe to CPU/GPU mode and load the net from disk

In [3]:
caffe.set_device(0)
caffe.set_mode_cpu()

model_def = '../../../models/SSD_300x300/test.prototxt'
model_weights = '../../../models/SSD_300x300/VGG_VOC0712_SSD_300x300_iter_120000.caffemodel'

net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

In [4]:
### perform detection
output = net.forward()

In [5]:
# for each layer, show the output shape
for layer_name, blob in net.blobs.iteritems():
    print layer_name + '\t' + str(blob.data.shape)

data	(1, 3, 300, 300)
label	(1, 1, 2, 8)
data_data_0_split_0	(1, 3, 300, 300)
data_data_0_split_1	(1, 3, 300, 300)
data_data_0_split_2	(1, 3, 300, 300)
data_data_0_split_3	(1, 3, 300, 300)
data_data_0_split_4	(1, 3, 300, 300)
data_data_0_split_5	(1, 3, 300, 300)
data_data_0_split_6	(1, 3, 300, 300)
conv1_1	(1, 64, 300, 300)
conv1_2	(1, 64, 300, 300)
pool1	(1, 64, 150, 150)
conv2_1	(1, 128, 150, 150)
conv2_2	(1, 128, 150, 150)
pool2	(1, 128, 75, 75)
conv3_1	(1, 256, 75, 75)
conv3_2	(1, 256, 75, 75)
conv3_3	(1, 256, 75, 75)
pool3	(1, 256, 38, 38)
conv4_1	(1, 512, 38, 38)
conv4_2	(1, 512, 38, 38)
conv4_3	(1, 512, 38, 38)
conv4_3_relu4_3_0_split_0	(1, 512, 38, 38)
conv4_3_relu4_3_0_split_1	(1, 512, 38, 38)
pool4	(1, 512, 19, 19)
conv5_1	(1, 512, 19, 19)
conv5_2	(1, 512, 19, 19)
conv5_3	(1, 512, 19, 19)
pool5	(1, 512, 19, 19)
fc6	(1, 1024, 19, 19)
fc7	(1, 1024, 19, 19)
fc7_relu7_0_split_0	(1, 1024, 19, 19)
fc7_relu7_0_split_1	(1, 1024, 19, 19)
fc7_relu7_0_split_2	(1, 1024, 19, 19)
fc7_relu7

In [6]:
item_id = 0

In [7]:
conv4_3 = net.blobs['conv4_3_norm_mbox_priorbox'].data
fc7     = net.blobs['fc7_mbox_priorbox'].data
conv6_2 = net.blobs['conv6_2_mbox_priorbox'].data
conv7_2 = net.blobs['conv7_2_mbox_priorbox'].data
conv8_2 = net.blobs['conv8_2_mbox_priorbox'].data
conv9_2 = net.blobs['conv9_2_mbox_priorbox'].data

In [8]:
print 'conv4_3.shape={}, #priors={}'.format(conv4_3.shape, conv4_3.shape[2]/4)
print 'fc7.shape={}, #priors={}'.format(fc7.shape, fc7.shape[2]/4)
print 'conv6_2.shape={}, #priors={}'.format(conv6_2.shape, conv6_2.shape[2]/4)
print 'conv7_2.shape={}, #priors={}'.format(conv7_2.shape, conv7_2.shape[2]/4)
print 'conv8_2.shape={}, #priors={}'.format(conv8_2.shape, conv8_2.shape[2]/4)
print 'conv9_2.shape={}, #priors={}'.format(conv9_2.shape, conv9_2.shape[2]/4)

conv4_3.shape=(1, 2, 23104), #priors=5776
fc7.shape=(1, 2, 8664), #priors=2166
conv6_2.shape=(1, 2, 2400), #priors=600
conv7_2.shape=(1, 2, 600), #priors=150
conv8_2.shape=(1, 2, 144), #priors=36
conv9_2.shape=(1, 2, 24), #priors=6


In [ ]:
blob_name = 'conv7_2_mbox_priorbox'
num_type_anchor = 6

color_list = ['red', 'orange', 'yellow', 'green', 'cyan', 'blue']

prior_blob = net.blobs[blob_name].data
print prior_blob.shape
num_priors = prior_blob.shape[2] / 4
step_size = 1

fig2 = plt.figure()
plt.title('{}: #priors={}'.format(blob_name, num_priors), fontsize=25)
ax2 = fig2.add_subplot(111, aspect='equal')
ax2.set_ylim([-0.5, 1.5])
ax2.set_xlim([-0.5, 1.5])
ax2.grid(True)
for i in xrange(0, num_type_anchor, step_size):
    [xmin, ymin, xmax, ymax] = prior_blob[0, 0, i*4:i*4+4]
    x = xmin
    y = ymin
    h = ymax - ymin
    w = xmax - xmin
    ax2.add_patch(
        patches.Rectangle(
            (x, y),
            w,
            h,
            fill=False,      # remove background
            alpha=0.6,
            edgecolor=color_list[i % num_type_anchor],
            linewidth=2
        )
    )